In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import sys
import glob
from os import listdir
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
%load_ext autoreload
%autoreload 2

In [ ]:
#Keras Imports
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.recurrent import LSTM

In [ ]:
#PianoRoll
sys.path.append(os.getcwd()+"/../../PianoRoll/")
sys.path.append(os.getcwd()+"/../../")
from PianoRoll import PianoRoll
from Utils import Utils
from metrics import Metrics

In [ ]:
res_factor = 12
note_threshold = 0.1
#Composer paths
chord_dir = os.getcwd() + "/../../data/testData/chords/"
composition_dir = os.getcwd() + "/compositions/"
#Checkpoint paths
model_dir = os.getcwd() + "/checkpoints/model/"
weights_dir = os.getcwd() + "/checkpoints/weights/"

#Other constants
MIDI_FILE_EXTENSION = "mid"
NOTE_ON = "note_on"
NOTE_OFF = "note_off"
MAX_NOTE = "max_note"
MIN_NOTE = "min_note"
TICKS = "ticks"
MIN_NOTE_INITIALIZER = 10000
MAX_NOTE_INITIALIZER = 0
MAX_TICK_INITIALIZER = 0
MELODY_LOWEST_NOTE = 60

In [ ]:
chord_files = glob.glob("%s*.mid" %(chord_dir))
composition_files = []
for i in range(len(chord_files)):
    composition_files.append(composition_dir + 'generated_%d.mid' %(i+1))
print(chord_files, composition_files)

In [ ]:
chord_roll = PianoRoll(chord_dir, res_factor=res_factor)
print("# of Ticks:{}\tMax_Note:{}\tMin_Note:{}".format(chord_roll.ticks,
                                                  chord_roll.max_note,
                                                  chord_roll.min_note))

In [ ]:
X_roll = chord_roll.chord2roll_wrapper(chord_dir, 
                              chord_roll.ticks, 
                              chord_roll.min_note,
                              chord_roll.max_note - chord_roll.min_note + 2,
                              chord_roll.res_factor)
print(X_roll.shape)

In [ ]:
X_double_roll = np.tile(X_roll, (1, 2, 1))
X_test = chord_roll.create_test_inputs(X_double_roll, seq_length=chord_roll.ticks)
print("Test Data:", X_test.shape)

In [ ]:
# chord_test_data = chord_test_roll.generate_piano_roll_func()
# X_test = PianoRoll.generate_test_samples(chord_test_data, chord_test_roll.ticks)
# print(chord_test_data.shape, X_test.shape)

In [ ]:
#Load Model
model_files = listdir(model_dir)
print("Choose a file for the model:")
print("---------------------------------------")
for i, file in enumerate(model_files):
    print(str(i) + " : " + file)
print("---------------------------------------")
file_index = input("Enter the index of the file model that you want:\t")
model_path = model_dir + model_files[file_index]
print("Loading model from {}".format(model_path))
model = model_from_json(open(model_path).read())

In [ ]:
#Load Weights
weights_files = listdir(weights_dir)
print("Choose a file for the weights:")
print("---------------------------------------")
for i, file in enumerate(weights_files):
    print(str(i) + " : " + file)
print("---------------------------------------")
file_index = input("Enter the file index of the weights that you want")
weights_path = weights_dir + weights_files[file_index]
print("Loading weights from ", weights_path)
model.load_weights(weights_path)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
keys_count = {}
for i,song in enumerate(X_test):
    net_output = model.predict(song)
    keys_count = PianoRoll.roll2midi_wrapper(composition_files[i], net_output, 60, 12, keys_count)
#     net_roll = PianoRoll.NetOutToPianoRoll(net_output, threshold=note_threshold)
#     PianoRoll.createMidiFromPianoRoll(net_roll, MELODY_LOWEST_NOTE, composition_dir,
#                                                composition_files[i], note_threshold, res_factor=res_factor)
    print("Finished composing song {}.".format(composition_files[i]))
print("Piano Key Distribution:{}".format(keys_count))